In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import datetime as dt
import math

# 시각화 관련
import plotly_express as px
import plotly.figure_factory as ff
import cufflinks as cf
cf.go_offline(connected=True)
## cf.getThemes()
cf.set_config_file(theme='polar')
import plotly.graph_objects as go
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
## plt.style.use("fivethirtyeight")
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams["figure.figsize"] = (10,6)


## 퀀트함수 ## 절대경로 참조
import sys, os
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))
from library.quant_functions import anal_funcs, data_funcs

In [9]:
data_ori = data_funcs.get_merged_df('^IXIC')
data_ori = pd.DataFrame(data_ori)

[*********************100%***********************]  1 of 1 completed


In [70]:
ticker = 'Close'
mdd_df = data_ori.copy()
mdd_df['hp'] = [mdd_df[ticker][:i+1].max() for i in range(len(mdd_df[ticker]))]
mdd_df['mdd'] = (mdd_df[ticker] - mdd_df['hp'])/mdd_df['hp']

In [72]:
mdd_df

,Close,hp,mdd
Date,,,
1971-02-05,100.000000,100.000000,0.000000
1971-02-08,100.839996,100.839996,0.000000
1971-02-09,100.760002,100.839996,-0.000793
1971-02-10,100.690002,100.839996,-0.001487
1971-02-11,101.449997,101.449997,0.000000
...,...,...,...
2023-07-19,14358.019531,16057.440430,-0.105834
2023-07-20,14063.309570,16057.440430,-0.124187
2023-07-21,14032.809570,16057.440430,-0.126087


In [65]:

rets_df = data_ori.pct_change().dropna()
rets_df['signal'] = (mdd_df['mdd'] < 0).shift(1)
rets_df['lev'] = 1
rets_df['lev'][rets_df['signal']==True] = 3
rets_df['perform'] = rets_df['Close'] * rets_df['lev']
rets_df['perform'] = (rets_df['perform'] + 1).cumprod()

In [66]:
result = data_ori.copy()
result['perform'] = rets_df['perform'].copy()
result['perform'][0] = 1
result = result/np.array(result)[0]